this notebook is responsible for updating new covid-19 dataset and saving on SQLite3 database.

In [ ]:
import json
import pandas as pd
from tqdm import tqdm

fetch data from nytime repo.

In [114]:
covid19_data_url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"

data = pd.read_csv(covid19_data_url)

In [115]:
data = data[['date', 'state', 'cases', 'deaths']]
data = data.rename(columns={"state": "states"})

In [116]:
data

,date,states,cases,deaths
0,2020-01-21,Washington,1,0
1,2020-01-22,Washington,1,0
2,2020-01-23,Washington,1,0
3,2020-01-24,Illinois,1,0
4,2020-01-24,Washington,1,0
...,...,...,...,...
24649,2021-05-24,Virginia,673105,11116
24650,2021-05-24,Washington,431446,5754
24651,2021-05-24,West Virginia,160354,2775
24652,2021-05-24,Wisconsin,672801,7749


table reshaping for visualization

In [117]:
data['number'] = data['cases']
data['cases'] = 'confirmed'

In [118]:
data

,date,states,cases,deaths,number
0,2020-01-21,Washington,confirmed,0,1
1,2020-01-22,Washington,confirmed,0,1
2,2020-01-23,Washington,confirmed,0,1
3,2020-01-24,Illinois,confirmed,0,1
4,2020-01-24,Washington,confirmed,0,1
...,...,...,...,...,...
24649,2021-05-24,Virginia,confirmed,11116,673105
24650,2021-05-24,Washington,confirmed,5754,431446
24651,2021-05-24,West Virginia,confirmed,2775,160354
24652,2021-05-24,Wisconsin,confirmed,7749,672801


In [119]:
len_of_current_data = len(data)
data_list = data.values.tolist()


In [120]:
for each in tqdm(data_list[:len_of_current_data]):
    data_list.append(
        [
            each[0], each[1], 'deaths', each[3], each[3]
        ]
    )

100%|██████████| 24654/24654 [00:00<00:00, 1598194.35it/s]


In [121]:
data = pd.DataFrame(data=data_list, columns=list(data.columns))

In [122]:
data

,date,states,cases,deaths,number
0,2020-01-21,Washington,confirmed,0,1
1,2020-01-22,Washington,confirmed,0,1
2,2020-01-23,Washington,confirmed,0,1
3,2020-01-24,Illinois,confirmed,0,1
4,2020-01-24,Washington,confirmed,0,1
...,...,...,...,...,...
49303,2021-05-24,Virginia,deaths,11116,11116
49304,2021-05-24,Washington,deaths,5754,5754
49305,2021-05-24,West Virginia,deaths,2775,2775
49306,2021-05-24,Wisconsin,deaths,7749,7749


In [123]:
new_col = list(data.columns)
new_col.remove('deaths')

In [124]:
data = data[new_col]
data

,date,states,cases,number
0,2020-01-21,Washington,confirmed,1
1,2020-01-22,Washington,confirmed,1
2,2020-01-23,Washington,confirmed,1
3,2020-01-24,Illinois,confirmed,1
4,2020-01-24,Washington,confirmed,1
...,...,...,...,...
49303,2021-05-24,Virginia,deaths,11116
49304,2021-05-24,Washington,deaths,5754
49305,2021-05-24,West Virginia,deaths,2775
49306,2021-05-24,Wisconsin,deaths,7749


In [125]:
data = data.sort_values(['date', 'states', 'cases']).reset_index(drop=True)
data 

,date,states,cases,number
0,2020-01-21,Washington,confirmed,1
1,2020-01-21,Washington,deaths,0
2,2020-01-22,Washington,confirmed,1
3,2020-01-22,Washington,deaths,0
4,2020-01-23,Washington,confirmed,1
...,...,...,...,...
49303,2021-05-24,West Virginia,deaths,2775
49304,2021-05-24,Wisconsin,confirmed,672801
49305,2021-05-24,Wisconsin,deaths,7749
49306,2021-05-24,Wyoming,confirmed,59797


save the dataset into SQLite3 databse

In [131]:
import sqlite3
import os

In [132]:
if not os.path.exists('./dataset/database/covid19'):
    os.makedirs('./dataset/database/covid19')

In [133]:
conn = sqlite3.connect('./dataset/database/covid19/covid19.sqlite')

In [134]:
data.to_sql('us_states', conn, if_exists='replace', index=False)

In [135]:
pd.read_sql('select * from us_states', conn)

,date,states,cases,number
0,2020-01-21,Washington,confirmed,1
1,2020-01-21,Washington,deaths,0
2,2020-01-22,Washington,confirmed,1
3,2020-01-22,Washington,deaths,0
4,2020-01-23,Washington,confirmed,1
...,...,...,...,...
49303,2021-05-24,West Virginia,deaths,2775
49304,2021-05-24,Wisconsin,confirmed,672801
49305,2021-05-24,Wisconsin,deaths,7749
49306,2021-05-24,Wyoming,confirmed,59797
